Cortical surface with subcortical regions, sEEG, EEG & MEG, using a stochastic
integration.

In [ ]:

from pylab import *
from tvb.simulator.lab import *
from tvb.datatypes.cortex import Cortex
from tvb.datatypes.region_mapping import RegionMapping
from tvb.datatypes.projections import ProjectionMatrix


oscillator = models.Generic2dOscillator()
white_matter = connectivity.Connectivity.from_file('connectivity_192.zip')
white_matter.speed = numpy.array([4.0])
white_matter_coupling = coupling.Difference(a=0.014)
heunint = integrators.HeunStochastic(
    dt=2**-4,
    noise=noise.Additive(nsig=numpy.array([2 ** -10, ]))
)
fsamp = 1e3/1024.0 # 1024 Hz
monitors = (
    monitors.EEG.from_file('eeg-brainstorm-65.txt', 'projection_EEG_surface.npy', period=fsamp),
    monitors.MEG.from_file('meg-brainstorm-276.txt', 'projection_MEG_surface.npy', period=fsamp),
    monitors.iEEG.from_file('SEEG_588.txt', 'projection_SEEG_surface.npy', period=fsamp),
)
local_coupling_strength = numpy.array([2 ** -10])
default_cortex = Cortex(region_mapping_data=RegionMapping.from_file('regionMapping_16k_192.txt'),
                        load_default=True)
default_cortex.coupling_strength = local_coupling_strength
sim = simulator.Simulator(model=oscillator, connectivity=white_matter,
                          coupling=white_matter_coupling,
                          integrator=heunint, monitors=monitors,
                          surface=default_cortex)
sim.configure()

ts, ys = {}, {}
mons = 'eeg meg seeg'.split()
for key in mons:
    ts[key] = []
    ys[key] = []

for data in sim(simulation_length=2**2):
    for key, dat in zip(mons, data):
        if dat:
            t, y = dat
            ts[key].append(t)
            ys[key].append(y)

figure()
for i, key in enumerate(mons):
    ts[key] = numpy.array(ts[key])
    ys[key] = numpy.array(ys[key])
    subplot(3, 1, i + 1)
    plot(ts[key], ys[key][:, 0, :, 0], 'k')
    title(key)
tight_layout()
show()